## 简答题
1. 如果训练集有100万个实例，训练决策树（无约束）大致的深度是多少？

20

2. 通常来说，子节点的基尼杂质是高于还是低于其父节点？是通常更高/更低？还是永远更高/更低？

一个节点的基尼不纯度通常比其父节点低

3. 如果决策树过拟合训练集，减少max_depth是否为一个好主意？

是一个好主意，降低max_depth可以降低模型复杂度

4. 如果决策树对训练集欠拟合，尝试缩放输入特征是否为一个好主意？

不是好主意，缩放特征不会影响决策树的分割结果

5. 如果在给定的训练集上训练决策树需要一个小时，那么如果将特征数量变为两倍，训练大约需要多少时间？

2小时

6. 如果在包含100万个实例的训练集上训练决策树需要一个小时，那么在包含1000万个实例的训练集上训练决策树，大概需要多长时间？提示：考虑CART算法的计算复杂度。

10-15小时

## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier

X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

dt = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_leaf_nodes': [None, 10, 20, 30, 50, 100, 200],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_params_, grid_search.best_score_, grid_search.best_estimator_

({'max_depth': None,
  'max_leaf_nodes': 10,
  'min_samples_leaf': 1,
  'min_samples_split': 2},
 0.854875,
 DecisionTreeClassifier(max_leaf_nodes=10, random_state=42))

In [18]:
from sklearn.metrics import accuracy_score

best_dt = grid_search.best_estimator_
best_dt.fit(X_train, y_train)
y_pred = best_dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.851

2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [19]:
from scipy.stats import mode
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(n_splits=1000, train_size=100, random_state=42)
X_train_subsets = []
y_train_subsets = []
for train_index, test_index in shuffle_split.split(X_train):
    X_train_subsets.append(X_train[train_index])
    y_train_subsets.append(y_train[train_index])
y_pred = []
for i in range(1000):
    best_dt.fit(X_train_subsets[i], y_train_subsets[i])
    y_pred.append(best_dt.predict(X_test))
result = mode(y_pred, axis=0)
accuracy = accuracy_score(y_test, result[0])
accuracy

0.8575